In [44]:
from transformers import BertModel,AutoTokenizer, FlaubertModel, BertForSequenceClassification , FlaubertForSequenceClassification
from transformers.modeling_utils import SequenceSummary
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.nn.utils.rnn import pack_padded_sequence
import sys
import re
from easy_nlp.models import TwoTaskLearning
from easy_nlp.training import Train
from easy_nlp.preprocessing import TextPreprocessing
from easy_nlp.feature_extraction import MetaFeaturesExtraction
from easy_nlp.data_visualisation import WordCloudMaker
from transformers import  AutoModel


In [45]:
import pandas as pd
df = pd.read_csv("/data/aboumada/Data/3_Datasets/corpus_PSY_Prepared_Features.csv")
#df_en = pd.read_csv('/data/aboumada/Data/3_Datasets/anonym_final_en_1.csv')
len(df)

3242

In [46]:
from easy_nlp.preprocessing import TextPreprocessing
tp=TextPreprocessing(df,'TEXT')
tp.fit_transform()

## Run if Random Sampling 

In [47]:
from sklearn.model_selection import train_test_split
df_train , df_test = train_test_split(df,random_state=1, test_size=0.2)


In [49]:
def get_sentences_labels(df,text_column='text_clean',label_column='CAT'):    
    dic_labels_to_cat = {value:x for x,value in enumerate(df[label_column].unique())}
    dic_cat_labels = {x:value for x,value in enumerate(df[label_column].unique())}
    df['text_clean']= df['text_clean'].map(lambda text_clean : re.sub('["#$%&()*+,-./:;<=>@[\]^_`{|}~\n\t’\']', '', text_clean))
    df2 = df[label_column].map(dic_labels_to_cat)
    sentences = df[text_column].values
    labels = df2.values.astype(int)
    return sentences,labels,dic_cat_labels

def custom_sentences_labels(df,dic_cat,dic_cat2,text_column='text_clean',label_column='CAT'):
    dic_cat_labe = dic_cat
    dic_cat_labe2 = dic_cat2
    #df['text_clean']= df['text_clean'].map(lambda text_clean : re.sub('["#$%&()*+,-./:;<=>@[\]^_`{|}~\n\t’\']', '', text_clean))
    dic_labels_to_cat = {v: k for k, v in dic_cat_labe.items()}
    dic_labels_to_cat2 = {v: k for k, v in dic_cat_labe2.items()}
    sentences = df[text_column].values
    df_cat = df['classe1'].map(dic_labels_to_cat)
    labels_CAT = df_cat.values.astype(int)
    df_cat2 = df['classe2'].map(dic_labels_to_cat2)
    labels_CAT2 = df_cat2.values.astype(int)
    
    return sentences,(labels_CAT,labels_CAT2),dic_cat_labe

dic_cat_labels_CAT = {0: 'opinionNegative', 1: 'sansOpinion-ou-mixte', 2: 'Poubelle', 3: 'opinionPositive'}
dic_cat_labels_CAT2 = {0: 'UsageMedical', 1: 'UsageDetourne', 2: 'Poubelle'}


#sentences_train,labels_train,dic_cat_labels=get_sentences_labels(df_train,text_column='text_clean',label_column='CAT')
sentences_train,labels_train,_=custom_sentences_labels(df_train,dic_cat_labels_CAT,dic_cat_labels_CAT2,text_column='processed_text',label_column='CAT')
sentences_test,test_labels,_=custom_sentences_labels(df_test,dic_cat_labels_CAT,dic_cat_labels_CAT2,text_column='processed_text',label_column='CAT')

In [50]:
from sklearn.model_selection import train_test_split
from easy_nlp.feature_extraction import BertInput

bert_input= BertInput(AutoTokenizer.from_pretrained('flaubert/flaubert_base_cased'))


X_train = bert_input.fit_transform(sentences_train)
X_test = bert_input.fit_transform(sentences_test)


In [51]:
# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels_cat, validation_labels_cat,train_labels_cat2, validation_labels_cat2 = train_test_split(X_train[0], labels_train[0], labels_train[1],
                                                            random_state=1, test_size=0.2)
# Do the same for the masks.
train_masks, validation_masks= train_test_split(X_train[1],
                                             random_state=1, test_size=0.2)

test_inputs = X_test[0]
test_masks = X_test[1]
test_labels_cat = test_labels[0]
test_labels_cat2 = test_labels[1]


In [52]:
import torch 
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
test_inputs = torch.tensor(test_inputs)

train_labels_cat = torch.tensor(train_labels_cat)
validation_labels_cat = torch.tensor(validation_labels_cat)
test_labels_cat = torch.tensor(test_labels_cat)



train_labels_cat2 = torch.tensor(train_labels_cat2)
validation_labels_cat2 = torch.tensor(validation_labels_cat2)
test_labels_cat2 = torch.tensor(test_labels_cat2)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
test_masks = torch.tensor(test_masks)



In [54]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs,train_masks,train_labels_cat,train_labels_cat2)
train_sampler = RandomSampler(train_data)
#train_sampler = ImbalancedDatasetSampler(train_data,callback_get_label=get_label_callback)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs,validation_masks ,validation_labels_cat,validation_labels_cat2)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs,test_masks, test_labels_cat,test_labels_cat2)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [55]:
model = TwoTaskLearning(model=AutoModel.from_pretrained('flaubert-base-cased'),
        dropout_rate = 0.1, 
        device = torch.device("cuda"),
        )
model.cuda()

TwoTaskLearning(
  (bert): FlaubertModel(
    (position_embeddings): Embedding(512, 768)
    (embeddings): Embedding(68729, 768, padding_idx=2)
    (layer_norm_emb): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (attentions): ModuleList(
      (0): MultiHeadAttention(
        (q_lin): Linear(in_features=768, out_features=768, bias=True)
        (k_lin): Linear(in_features=768, out_features=768, bias=True)
        (v_lin): Linear(in_features=768, out_features=768, bias=True)
        (out_lin): Linear(in_features=768, out_features=768, bias=True)
      )
      (1): MultiHeadAttention(
        (q_lin): Linear(in_features=768, out_features=768, bias=True)
        (k_lin): Linear(in_features=768, out_features=768, bias=True)
        (v_lin): Linear(in_features=768, out_features=768, bias=True)
        (out_lin): Linear(in_features=768, out_features=768, bias=True)
      )
      (2): MultiHeadAttention(
        (q_lin): Linear(in_features=768, out_features=768, bias=True)
       

In [56]:
from transformers import AdamW,get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 3e-8 # args.adam_epsilon  - default is 1e-8.
                )


# Number of training epochs (authors recommend between 2 and 4)
epochs = 1

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs 

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [57]:
import numpy as np
train = Train()
train.fit_TwoTask(model,train_dataloader,validation_dataloader,epochs,torch.device('cuda'),optimizer,scheduler)
print("")
print("Training complete!")


======== Epoch 1 / 1 ========
Training...


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.91 GiB total capacity; 4.98 GiB already allocated; 11.06 MiB free; 5.15 GiB reserved in total by PyTorch)

In [58]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(test_dataloader)))

# Put model in evaluation mode
model.eval()


# Tracking variables 
predictions_cat,predictions_cat3,predictions_cat2 , true_labels_cat,true_labels_cat3,true_labels_cat2  = [], [],[],[],[],[]

# Predict 
for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(torch.device("cuda")) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask,b_labels_cat,b_labels_cat2 = batch

    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model((b_input_ids,b_input_mask))
        logits_cat = outputs[0]
        logits_cat2 = outputs[1]

    # Move logits and labels to CPU
    logits_cat = logits_cat.detach().cpu().numpy()
    label_ids_cat = b_labels_cat.to('cpu').numpy()
    predictions_cat.extend(logits_cat)
    true_labels_cat.extend(label_ids_cat)

    
    
    logits_cat2 = logits_cat2.detach().cpu().numpy()
    label_ids_cat2 = b_labels_cat2.to('cpu').numpy()
    predictions_cat2.extend(logits_cat2)
    true_labels_cat2.extend(label_ids_cat2)


print('    DONE.')



Predicting labels for 41 test sentences...


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.91 GiB total capacity; 4.98 GiB already allocated; 11.06 MiB free; 5.15 GiB reserved in total by PyTorch)

In [59]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

pred_flat_cat = np.argmax(predictions_cat, axis=1)
true_labels_cat=[dic_cat_labels_CAT.get(x) for x in true_labels_cat]
pred_flat_cat = [dic_cat_labels_CAT.get(x) for x in pred_flat_cat]


cr= classification_report(true_labels_cat,pred_flat_cat,digits=4)
print(accuracy_score(pred_flat_cat,true_labels_cat))
print(cr)

AxisError: axis 1 is out of bounds for array of dimension 1

In [38]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

pred_flat_cat2 = np.argmax(predictions_cat2, axis=1)
true_labels_cat2 =[dic_cat_labels_CAT2.get(x) for x in true_labels_cat2]
pred_flat_cat2 = [dic_cat_labels_CAT2.get(x) for x in pred_flat_cat2]


cr= classification_report(true_labels_cat2,pred_flat_cat2,digits=4)
print(accuracy_score(pred_flat_cat2,true_labels_cat2))
print(cr)

ValueError: Classification metrics can't handle a mix of unknown and binary targets

In [ ]:
model.save('model.pth')